# Contas Econômicas: Investigando os Detalhes da Economia Brasileira

----

Vamos, usando os dados disponíveis no [Sistema de Contas Nacionais do IBGE](https://www.ibge.gov.br/estatisticas/economicas/contas-nacionais/9052-sistema-de-contas-nacionais-brasil.html?=&t=downloads), analisar a economia brasileira ao nível do **produto**, usando, para isso, as **Tabelas de Recursos e Usos** (TRUs, para os íntimos).

Como vamos fazer análises com valores monetários que variam no tempo, também vamos aprender um pouco sobre como deflacionar esses valores para permitir comparações em termos reais.

## Bibliotecas e Módulos

In [ ]:
## Importando o que for necessário
# Manipulação de dados
import pandas as pd
import numpy as np

# Testes e regressões
from scipy import stats
import statsmodels.api as sm
import statsmodels.stats.api as sms

# Gráficos
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
from IPython.display import clear_output # limpa o output de uma célula

style.use('ggplot')  # estilo do R

In [ ]:
### Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Definindo o caminho e o arquivo
# Caminho
sCaminho = "/content/drive/Othercomputers/Meu modelo MacBook Air/Documents/IPEA/Curso IDP/Disciplina 3 - Crescimento Econômico/Dados/"
# sCaminho = "/Users/vinicius/Documents/IPEA/Curso IDP/Disciplina 3 - Crescimento Econômico/Dados/"

# Arquivo a preços do ano corrente
sArquivo = "Compilacao_Tru_Retro.xlsx"

# Arquivo a preços do ano anterior
sArquivoPA = "Compilacao_Tru_RetroPA.xlsx"

# Nome da aba
sAbaInicial = "OfertaNacPB"

## Deflação - Exemplo da Produção

In [ ]:
## Lendo o DataFrame a Preços Correntes
# Usaremos a Oferta Nacional a Preços Básicos como Produção
# Oferta nacional: só o que foi produzido no país (sem exportações e importações)
# PB: preços básicos/preço de fábrica (sem impostos e margens de comércio e transporte)
dfProducao = pd.read_excel(f"{sCaminho}{sArquivo}", sheet_name=sAbaInicial, index_col=0)

## Preços do Ano Anterior
dfProducaoPA = pd.read_excel(f"{sCaminho}{sArquivoPA}", sheet_name=sAbaInicial, index_col=0)

In [ ]:
## Vendo os DataFrames
# Preços Correntes
dfProducao.head(5)

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Arroz em casca,2114.751756,2451.561130,3125.120754,4815.610028,6393.552924,4571.130475,3922.697755,4163.971457,6375.204893,6453.815925,5228,5009,5400,6517,7226,7519,7560,8488,7503,7465
Milho em grão,6012.728095,6288.109714,8806.456861,14495.149537,12467.198444,10065.314358,10589.551454,16687.866489,22271.717346,16138.461607,14717,22496,26568,26861,26008,29975,33198,30269,34658,43663
Trigo em grão e outros cereais,534.887097,1047.766317,1401.806077,2878.319580,2643.229557,1851.078395,1366.402101,2382.586272,3463.141035,2601.841710,2814,3417,3296,4983,3528,3517,4613,3064,4673,5166
Cana-de-açúcar,10308.631923,11932.112877,13136.992226,16417.754233,16238.600467,17573.494867,23585.909075,23332.108922,24250.289287,28971.979887,32894,40949,41940,44863,43515,46080,54151,55359,53786,57068
Soja em grão,9480.377318,13568.333119,21118.082882,33624.538827,38382.586396,26589.972807,22584.136218,30186.126774,43906.109538,42686.778694,40124,52217,61520,84057,90768,109170,119120,128214,157189,153331


In [ ]:
# Preços do Ano Anterior
dfProducaoPA.head(5)

,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Arroz em casca,1932.464807,2513.709958,3091.615658,6186.511642,6290.307257,3994.628437,3763.254840,4542.604994,6686.769113,5809.338936,6403,4225,5553,6740,7331,6486,8884,8115,6543
Milho em grão,7806.233616,5385.696691,11949.760075,12536.930675,10382.762504,12341.822011,13050.711277,18871.991645,19177.428444,17519.358762,15456,27925,30553,27322,27137,22132,52123,25318,42710
Trigo em grão e outros cereais,872.660165,944.424981,2695.358215,2837.434359,2126.031508,1094.652663,2031.995499,3492.695424,2876.275319,2969.160540,2832,2779,4311,4964,3230,4292,3576,3683,4957
Cana-de-açúcar,10884.076581,12630.934518,14275.999824,17214.853305,16539.736797,19834.951937,25682.082485,26275.914537,26008.085157,30028.382700,31976,39830,44770,43378,43842,46910,53186,55016,54763
Soja em grão,10949.164540,15071.644822,26039.411222,31708.468758,39647.532442,27255.524370,24907.947088,31172.486760,42079.042543,51417.916524,43750,45239,77559,88843,102343,107118,142416,132622,151955


Note que não temos os dados a preços do ano anterior para 2000. Por isso, ele será nosso ano base e terá índice = 100.

Assim, quando terminarmos de deflacionar, tudo estará em R$ Milhões de 2000.

### Tabela de Índice de Preços

Um índice de preços é nada mais que um indicador que mede o quanto os preços variaram em relação a um período base.

Por exemplo, tomando 2000 = 100, um índice de 110 em 2003 indica que o preço variou 10% entre 2000 e 2003.

In [ ]:
## Anos que usaremos
lAnos = np.arange(2000, 2020)

## Número de Produtos
nProdutos = 107

## Matriz que armazenará os resultados
mIndicePreco = np.zeros((nProdutos, len(lAnos)), dtype=float)

## 2000 = 100
mIndicePreco[:, 0] = 100

In [ ]:
## Retirando coluna de 2000 do df a preços correntes
dfProducao2001 = dfProducao.drop(["2000"], axis=1)

## Dividindo a produção a preços correntes pela de preços do ano anterior para captar a variação de preços
dfVarPrecos = dfProducao2001 / dfProducaoPA
dfVarPrecos.head(5)

# Por exemplo, o preços de arroz em casca teria variado 26,8% entre 2000 e 2001!
# Não podemos dizer que o preço variou 24,32% entre 2000 e 2002, porque ainda precisamos
## encadear os índices (o certo é que o preço variou 1,2686 x 1,2432 - 1 = 57,71%!)

,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Arroz em casca,1.268619,1.243230,1.557635,1.033467,0.726694,0.981993,1.106481,1.403425,0.965162,0.899930,0.782290,1.278107,1.173600,1.072107,1.025645,1.165587,0.955425,0.924584,1.140914
Milho em grão,0.805524,1.635156,1.213008,0.994438,0.969425,0.858022,1.278694,1.180147,0.841534,0.840042,1.455487,0.951406,0.879161,0.951907,1.104580,1.500000,0.580723,1.368907,1.022313
Trigo em grão e outros cereais,1.200658,1.484296,1.067880,0.931556,0.870673,1.248252,1.172535,0.991538,0.904587,0.947743,1.206568,1.186038,1.155880,0.710717,1.088854,1.074790,0.856823,1.268803,1.042163
Cana-de-açúcar,1.096291,1.040065,1.150025,0.943290,1.062501,1.189108,0.908498,0.922909,1.113961,1.095430,1.280617,1.052975,1.002077,1.003158,1.051047,1.154359,1.040857,0.977643,1.042090
Soja em grão,1.239212,1.401180,1.291294,1.210484,0.670659,0.828608,1.211907,1.408489,1.014443,0.780351,1.193531,1.359889,1.083781,1.021667,1.066707,1.112045,0.900278,1.185241,1.009055


In [ ]:
## Checando NANs, 0s e INFs e substituindo por 1 (caso contrário, isso quebraria o encadeamento)
# Colocamos <= 0 porque, no caso de impostos, podem haver valores negativos (subsídios)
mVarPrecos = np.array(np.where(
    (np.isinf(dfVarPrecos)) | (np.isnan(dfVarPrecos)) | (dfVarPrecos <= 0), 
    1, 
    dfVarPrecos
))

In [ ]:
## Como dito anteriormente, precisamos encadear os índices
# Loopando sobre cada coluna de mVarPrecos
for nColuna in range(mVarPrecos.shape[1]):
  mIndicePreco[:, nColuna + 1] = mIndicePreco[:, nColuna] * mVarPrecos[:, nColuna]

In [ ]:
## Criando um DataFrame
dfIndicePreco = pd.DataFrame(mIndicePreco, index=dfProducao.index, columns=dfProducao.columns)
dfIndicePreco

# Assim, por exemplo, o preço do arroz em casca aumentou 145% entre 2000 e 2003; se não deflacionarmos,
# nossas análises irão captar essa variação de preço como se fosse um aumento da produção, o que não é desejável

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Arroz em casca,100.0,126.861877,157.718548,245.667996,253.889659,184.500170,181.177903,200.469978,281.344556,271.543095,244.369853,191.167982,244.333043,286.749224,307.425800,315.309588,367.520889,351.138823,324.657374,370.406128
Milho em grão,100.0,80.552415,131.715804,159.772268,158.883592,154.025800,132.157483,168.988983,199.431779,167.828660,140.983150,205.199078,195.227541,171.636402,163.381873,180.468425,270.702638,157.203118,215.196527,219.998267
Trigo em grão e outros cereais,100.0,120.065789,178.213153,190.310291,177.284731,154.357042,192.676448,225.919919,224.008237,202.634974,192.045802,231.716280,274.824346,317.664049,225.769292,245.829907,264.215602,226.386075,287.239242,299.349994
Cana-de-açúcar,100.0,109.629079,114.021362,131.127397,123.691174,131.422055,156.275077,141.975524,131.030550,145.962859,159.892137,204.760543,215.607762,216.055640,216.738005,227.801818,262.965172,273.709039,267.589690,278.852664
Soja em grão,100.0,123.921173,173.635833,224.214931,271.408533,182.022564,150.825290,182.786294,257.452512,261.170829,203.804803,243.247437,330.789415,358.503409,366.271258,390.704135,434.480447,391.153213,463.610732,467.808872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Serviços associativos,100.0,100.838218,115.607843,127.945469,144.504623,159.339975,170.519551,168.127051,177.998126,189.452551,199.446642,216.535189,230.525081,254.118327,263.420931,282.412452,310.209184,332.530848,362.331432,370.833858
Serviços domésticos,100.0,108.952935,113.164785,128.971859,139.505114,151.316207,169.368288,186.053281,206.513116,231.567158,254.845807,277.150584,306.146072,339.911717,366.638697,383.133020,413.681389,432.081338,446.604022,444.180937
Educação pública,100.0,103.119728,119.283528,128.874760,128.602206,148.117704,169.082036,202.902408,243.502555,276.732259,313.427151,384.170405,421.565085,521.698956,594.682632,652.039828,696.701446,720.650333,755.684917,833.021166
Saúde pública,100.0,104.389486,118.549705,131.398354,156.109715,162.998471,175.093884,187.554370,202.560300,219.191199,235.818907,252.913599,272.129088,309.453803,337.502088,356.509501,378.475215,370.731477,385.564853,407.214329


In [ ]:
## Para ter a produção a preços de 2000, basta dividir dfProducao pelos índices e multiplicar por 100
dfProducaoDef = dfProducao / dfIndicePreco * 100
dfProducaoDef

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Arroz em casca,2114.751756,1932.464807,1981.454171,1960.210572,2518.240774,2477.575208,2165.108267,2077.104757,2265.977694,2376.718853,2139.380102,2620.208644,2210.098128,2272.717575,2350.485876,2384.640459,2057.025937,2417.277569,2311.051776,2015.355427
Milho em grão,6012.728095,7806.233616,6685.953088,9072.381382,7846.750125,6534.823628,8012.827740,9875.120944,11167.586945,9616.034372,10438.836126,10963.012242,13608.735635,15649.943536,15918.534577,16609.553708,12263.641123,19254.707098,16105.278480,19846.974548
Trigo em grão e outros cereais,534.887097,872.660165,786.589573,1512.435068,1490.951615,1199.218623,709.169241,1054.615405,1545.988258,1284.004268,1465.275459,1474.648223,1199.311505,1568.638318,1562.657157,1430.664007,1745.922638,1353.440137,1626.866849,1725.739133
Cana-de-açúcar,10308.631923,10884.076581,11521.518444,12520.460731,13128.342142,13371.800363,15092.559535,16433.895240,18507.355177,19848.871235,20572.618921,19998.481870,19451.989862,20764.558563,20077.235614,20228.109015,20592.460805,20225.492057,20100.176503,20465.287729
Soja em grão,9480.377318,10949.164540,12162.283870,14996.565402,14141.996950,14608.064112,14973.706453,16514.436703,17054.061401,16344.389944,19687.465382,21466.618743,18597.934874,23446.638994,24781.633274,27941.859402,27416.653801,32778.460105,33905.384248,32776.419873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Serviços associativos,29142.675596,29509.380678,31281.350116,28296.022013,29760.168781,30874.589066,32074.139747,31305.228715,32558.343233,34249.231403,33412.445218,31980.483280,32616.407632,31216.559949,30969.824499,29507.905741,25578.546416,23887.107130,24553.210744,25298.660839
Serviços domésticos,12116.657259,12134.290037,13233.892639,13280.637158,14163.987516,14597.242204,14879.502165,14742.425199,14631.857010,15948.657732,15826.825064,16032.439591,15471.372760,15785.275203,15864.119205,16181.325219,16504.972604,16538.089878,16670.472361,16920.582065
Educação pública,43925.538046,44693.949936,45965.404407,47114.346482,49093.796699,48984.872950,49278.542026,48096.095458,47602.984425,47460.072339,46951.899127,46670.174970,45717.733044,45112.607028,44519.544695,44252.511506,44474.430465,44557.392830,44421.820840,43767.315253
Saúde pública,25433.549985,26538.524604,28823.052581,30250.031513,29970.038855,30404.673960,32459.308923,35095.027795,37415.016310,39948.022583,42347.749430,43943.465415,44276.780825,46084.746313,48338.071344,49165.029192,49208.506294,51034.781761,51485.242567,52209.115601


### Loopando sobre todas as abas da planilha e gerando um Excel

Agora que fizemos para a produção, podemos fazer o mesmo procedimento para todas as abas

In [ ]:
## Definindo as abas
vNomes = ["Exportação", "Consumo do Governo", "Consumo das Famílias_ISFLSF",
          "Investimento_FBCF", "Impostos_Subsidios", "Importação", "OfertaNacPB",
          "mOfertaTotPB", "mOfertaTotPC"]

## Criando lista para armazenar os resultados deflacionados
vData = []

In [ ]:
## Para cada aba,
for sNome in vNomes:
  ## Leitura
  print(f"Deflacionando {sNome}...")
  # Preços Correntes
  dfPC = pd.read_excel(f"{sCaminho}{sArquivo}", sheet_name=sNome, index_col=0)
  ## Preços do Ano Anterior
  dfPA = pd.read_excel(f"{sCaminho}{sArquivoPA}", sheet_name=sNome, index_col=0)

  ## Matriz que armazenará os resultados
  mIndicePreco = np.zeros((nProdutos, len(lAnos)), dtype=float)

  ## 2000 = 100
  mIndicePreco[:, 0] = 100

  ## Retirando coluna de 2000 do df a preços correntes
  dfPC2001 = dfPC.drop(["2000"], axis=1)

  ## Dividindo a produção a preços correntes pela de preços do ano anterior para captar a variação de preços
  dfVarPrecos = dfPC2001 / dfPA

  ## Checando NANs, 0s e INFs e substituindo por 1 (caso contrário, isso quebraria o encadeamento)
  # Colocamos <= 0 porque, no caso de impostos, podem haver valores negativos (subsídios)
  mVarPrecos = np.array(np.where(
      (np.isinf(dfVarPrecos)) | (np.isnan(dfVarPrecos)) | (dfVarPrecos <= 0), 
      1, 
      dfVarPrecos
  ))

  # Loopando sobre cada coluna de mVarPrecos
  for nColuna in range(mVarPrecos.shape[1]):
    mIndicePreco[:, nColuna + 1] = mIndicePreco[:, nColuna] * mVarPrecos[:, nColuna]

  ## Criando um DataFrame
  dfIndicePreco = pd.DataFrame(mIndicePreco, index=dfPC.index, columns=dfPC.columns)

  ## Para ter a produção a preços de 2000, basta dividir dfProducao pelos índices e multiplicar por 100
  dfDef = dfPC / dfIndicePreco * 100

  ## Adicionando à lista
  vData.append(dfDef)

clear_output()
print("Acabou!")

Acabou!


In [ ]:
## Objeto Writer (salvar uma planilha com várias abas)
Writer = pd.ExcelWriter(f"{sCaminho}Compilacao_Tru_RetroDEF.xlsx", engine='openpyxl')
  
## Para cada DataFrame/aba
for nAba in range(len(vNomes)):
  vData[nAba].to_excel(Writer, vNomes[nAba], freeze_panes=(1, 1))

## Saving file
Writer.save()